In [36]:
from keras.models import Model
import pickle
import csv
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.misc import imread
from keras import optimizers
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D,Input,Cropping2D,ZeroPadding2D
from sklearn.utils import shuffle
from keras.models import Sequential,load_model
from keras.layers import Activation, Dropout, UpSampling2D, Lambda,Cropping2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D,Deconvolution2D,Reshape,Add
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.optimizers import SGD
import h5py
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import h5py

In [ ]:
#1.Output the lane lines mark video

In [28]:
def predictlane(model_path,model_input_size,videopath):
    #load the trained model
    global model
    net_model = load_model(model_path)
    model = Sequential()
    model.add(net_model)
    model.add(Activation('softmax'))
    #define inner function to draw lane on images
    def drawlane(image):
        image = np.asarray(image)
        original_imgsize = (1080,1920,3)
        highway_im = image
        #change the image to the model input size
        image = scipy.misc.imresize(image, model_input_size)
        image = image[np.newaxis,:] #increase the dimension
        #predict softmax graph
        softmax = model.predict([image])
        #downscale the output tensor dimension and mark the coresponding pixel in mask
        softmax_r = softmax[0][:,:,1].reshape(model_input_size[0], model_input_size[1])
        segmentation_r = (softmax_r > 0.5).reshape(model_input_size[0], model_input_size[1], 1)
        #define object color
        obj_color_schema = [[0, 255, 0, 127]]
        mask = np.dot(segmentation_r, np.array(obj_color_schema))
        #recover the mask back to cropped image size
        mask = scipy.misc.imresize(mask, (1080,1920,3))
        mask = mask[650:1080][:]
        mask = scipy.misc.toimage(mask, mode="RGBA")
        #combine the mask and the original image
        highway_im = scipy.misc.toimage(highway_im)
        highway_im.paste(mask, box=(0,649), mask=mask)
        highway_im = np.asarray(highway_im)
        return highway_im
    
    test_output = './segmarklane.mp4'
    clip1 = VideoFileClip(videopath).subclip(0,315)
    white_clip = clip1.fl_image(drawlane) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(test_output, audio=False)

In [19]:
model_path = './Lenetlanebeta1.h5'
videopath = './video 2_processed.mov'
model_input_size = (540,960,3)
predictlane(model_path,model_input_size,videopath)

[MoviePy] >>>> Building video ./segmarklane.mp4
[MoviePy] Writing video ./segmarklane.mp4


100%|██████████| 9441/9441 [1:11:43<00:00,  2.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./segmarklane.mp4 

CPU times: user 1h 55min 29s, sys: 4min 1s, total: 1h 59min 31s
Wall time: 1h 11min 46s
